In [ ]:
import time
import pandas as pd
from data.loader import load_price_data
from data.feed_source import FeedSource
from strategies.example_strategy import ExampleStrategy
from signal.generator import SignalGenerator
from order_execution.mock_broker import MockBroker
from order_execution.order_manager import OrderManager
from utils.reporter import generate_report

symbol = "AAPL"
start = "2023-01-01"
end = "2023-06-01"

feed = FeedSource(symbol, start=start, end=end)
feed.load()

broker = MockBroker()
order_manager = OrderManager(broker)

# 전체 데이터를 기반으로 시그널 먼저 생성
df = feed.data.copy()
strategy = ExampleStrategy(df)
strategy.generate_signals()

signal_generator = SignalGenerator()

# 실시간처럼 시뮬레이션
while feed.has_next():
    bar = feed.next()
    timestamp = bar.name

    entry, exit = signal_generator.extract_point(
        timestamp, strategy.entries, strategy.exits
    )

    if entry:
        print(f"[{timestamp}] 🟢 매수 시그널 → 주문 실행")
        order_manager.handle_signal(symbol, "buy", quantity=10)

    elif exit:
        print(f"[{timestamp}] 🔴 매도 시그널 → 주문 실행")
        order_manager.handle_signal(symbol, "sell", quantity=10)

    # sleep은 실제 상황에서는 0.1초~1초 단위로 맞춤
    time.sleep(0.01)

# 포지션/잔고/체결 결과 요약
report = generate_report(broker)

print("\n📊 최종 리포트:")
for k, v in report.items():
    print(f"{k}: {v}")
